### This notebook contains skeleton code, and discusses some ideas on how to approach the problem of designing a robot control architecuture using BLE.

# A Robot Control Class
Classes provide a means of bundling data and functionality together. It helps organize your thoughts and your code. 

Below is the skeleton of one possible class structure to deal with robot communication. You do not have to use the exact same structure, it merely serves as a reference.

In [ ]:
%load_ext autoreload
%autoreload 2

from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import time
import numpy as np
import matplotlib.pyplot as plt
import asyncio

LOG.propagate = False

In [ ]:
class RobotControl():
    # Initialize Function
    def __init__(self, ble):
        self.ble = ble
        
        # A variable to store the latest sensor value
        self.latest_tof_front_reading = None
        self.latest_tof_side_reading = None
        
        # A list to store the history of all the sensor values
        # Each item in the list is a tuple (value, time)
        # WARNING: The list could grow really fast; you need to deal with this accordingly.
        self.tof_readings = []
        self.tof2_readings = []
        
        self.imu_readings = []
        
        self.motor_readings = []
        
        # A variable to store the latest imu reading
        self.latest_imu_reading = None
        
        # Activate notifications (if required)
        self.setup_notify()
    
    # A function to activate various notifications (if required)
    def setup_notify(self):
        # Code to setup various notify events
        # Ex:
        # ble.start_notify(ble.uuid['RX_TOF2'], self.tof_callback_handler)
        # ble.start_notify(ble.uuid['RX_FANCY_SENSOR'], self.fancy_sensor_callback_handler)
        self.ble.start_notify(self.ble.uuid['RX_TOF1'], self.tof_callback_handler)
        self.ble.start_notify(self.ble.uuid['RX_TOF2'], self.tof2_callback_handler)
        self.ble.start_notify(self.ble.uuid['RX_IMU'], self.imu_callback_handler)
        self.ble.start_notify(self.ble.uuid['RX_MOTOR'], self.motor_callback_handler)
    
    def stop_notify(self, sensor):
        self.ble.stop_notify(ble.uuid[f'RX_{sensor}'])
        
    # An example function callback handler for storing the history of the tof sensor
    # Your callback handlers should perform minimal processing!
    # Do not add a receive_* function inside the callback handler, it defeats the purpose of BLE notify
    def tof_callback_handler(self, uuid, byte_array):
        # Append a tuple (value, time) to a list
        self.tof_readings.append( ( self.ble.bytearray_to_float(byte_array), time.time() ) )
        #self.update_tof_readings()
    
    def tof2_callback_handler(self, uuid, byte_array):
        # Append a tuple (value, time) to a list
        self.tof2_readings.append( ( self.ble.bytearray_to_float(byte_array), time.time() ) )
        #self.update_tof2_readings()
    
    def imu_callback_handler(self, uuid, byte_array):
        self.imu_readings.append( ( self.ble.bytearray_to_float(byte_array), time.time() ) )
        #self.update_imu_readings()
    
    def motor_callback_handler(self, uuid, byte_array):
        self.motor_readings.append( ( self.ble.bytearray_to_float(byte_array), time.time() ) )
    
    def update_imu_readings(self):
        if len(self.imu_readings) > 10000:
            self.imu_readings = self.imu_readings[-10000:]
    
    def update_tof_readings(self):
        if len(self.tof_readings) > 10000:
            self.tof_readings = self.tof_readings[-10000:]
    
    def update_tof2_readings(self):
        if len(self.tof2_readings) > 10000:
            self.tof2_readings = self.tof2_readings[-10000:]
    
    # An example function to fetch the front TOF sensor reading
    # Here we assume RX_TOF1 is a valid UUID defined in connection.yaml and
    # in the Arduino code as well
    def get_front_tof(self):
        self.latest_tof_front_reading = self.ble.receive_float(self.ble.uuid['RX_TOF2'])
        print(self.latest_tof_front_reading)
    
    def get_side_tof(self):
        self.latest_tof_side_reading = self.ble.receive_float(self.ble.uuid['RX_TOF1'])
        print(self.latest_tof_side_reading)
    
    # An example function to fetch the IMU readings as a string
    # Here we assume RX_IMU is a valid UUID defined in connection.yaml and
    # in the Arduino code as well
    def get_imu(self):
        self.ble.send_command(CMD.GET_IMU, '')
        self.latest_imu_reading = self.ble.receive_float(self.ble.uuid['RX_IMU'])
        
        print(self.latest_imu_reading)
    
    # A function to instruct the robot to move forward
    def move_forward(self, speed, forward, doPID = 0):
        # Code to move forward
        # Ex: 
        # Here we assume the command is defined in cmd_types.py and 
        # the Artemis is programmed to handle it accordingly
        # ble.send_command(CMD.MOVE_FORWARD, speed)
        
        """
        speed: two element list ([motor1Speed, motor2Speed])
        time: duration of robot movement in seconds
        forward:
        doPID: 0 when PID should be done and 1 otherwise
        """
        self.tof_readings = []
        self.tof2_readings = []
        
        self.ble.send_command(CMD.MOVE_FORWARD, f'{speed[0]}|{speed[1]}|{forward}|{doPID}')
    
    # A function to stop robot motion
    def stop(self):
        # Code to stop robot motion
        self.ble.send_command(CMD.STOP_ROBOT, '')
    
    # A function to instruct the robot to update PID constants
    def updatePID(self, setpoint, k_p, k_i, k_d):
        self.ble.send_command(CMD.UPDATE_PID, f'{setpoint}|{k_p}|{k_i}|{k_d}')

### You can now use such a class to control your robot

In [ ]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

# Instantiate RobotControl class
rc = RobotControl(ble)

In [ ]:
startingSpeed = 75
rc.updatePID(300, 0.35, 0.00001, 0.75)

In [ ]:
rc.move_forward([startingSpeed, startingSpeed], 0)
await asyncio.sleep(10)
rc.stop()

In [ ]:
# Move backward after a test
rc.move_forward([100, 100], 1, 1) # 0 to go forward and 1 not to do PID
await asyncio.sleep(3)
rc.stop()

In [ ]:
file = 'trial102_motor.txt'
with open(file, 'w') as f:
    for item in rc.tof_readings:
        f.write(str(item))
        f.write("\n")

In [ ]:
def plot(x):
    start_time = x[0][1]
    plt.plot([(x[i][1] - start_time) for i in range(len(x))], [x[i][0] for i in range(len(x))])
    plt.title('Front ToF Sensor Reading (mm) vs. Time (msec)')
    plt.xlim([0, 0.032])
plot(rc.tof2_readings)

In [ ]:
def processRunData(file):
    with open(file, 'r') as f:
        x = f.read().splitlines()
    
    calculate = False
    comma = x[0].index(",")
    oldSensorValue = float(x[0][1:comma])
    oldTimeValue = float(x[0][comma+2:-1])

    for val in x[1:]:

        comma = val.index(",")
        newSensorValue = float(val[1:comma])
        newTimeValue = float(val[comma+2:-1])

        #print(newTimeValue-oldTimeValue)

        if (newTimeValue - oldTimeValue) > 0.01:
            timeDelta = newTimeValue - oldTimeValue
            sensorDelta = newSensorValue - oldSensorValue
            
            speed = sensorDelta/(timeDelta * 1000) # convert mm/s to m/s
            if speed > 0:
                print(f'{speed} m/s')

            oldTimeValue = newTimeValue
            oldSensorValue = newSensorValue

In [ ]:
#files = ['trial60_15.txt', '300+2x.txt', '300+2x_2.txt', 'trial10.txt', 
files=[         'trial30.txt', 'trial31.txt', 'trial32.txt']

for file in files:
    print(file)
    processRunData(file)
    print("\n")

### Lab 7

***KF essentially interpolates ToF Sensor Data***

In [4]:
# A = [ 
#         [ 0    1   ]
#         [ 0   -d/m ]
#     ]
    
# B = [ 
#         0
#         1/m
#     ]

# C = [ 1 0 ]

***For these parts, plot the PWM step function (PWM vs. time), front ToF sensor distance vs. time and speed vs. time***

-Find d: At steady state (constant speed), we can find d = u / x dot

    -u is the PWM signal value
        -Since we're analying the unit step response, we scale this to 1 for now (we are supposed to use the max PWM value from lab 6)
    -x dot is the steady state speed
    
    ***d = 1 / steady state speed
    
    -Lecture example:
        -Steady state speed: 2000 mm/s 


-Find m:
    
    -Find the 90% rise time
    
    -m = (-d * t_0.9) / ln(1-0.9)
   
-dt: time between samples

***We did initial analysis by setting u=1, so later scale down the PWM signal value by dividing it by the max value
    -Ex: if the max was 120, then one of the KF input arguments should be motor PWM/120***

In [ ]:
def calcRiseTime(data, start, max_, percent = 90):
    startTime = data[0][1]
    topValue = max_ * percent / 100
    for val in data: # data format - [ (motorPWM1, time1), (motorPWM2, time2) ]
        # Get the PWM signal
        if val[0] >= topValue:
            return val[1] - startTime

In [5]:
# u is the input (PWM signal)
# x dot is velocity
# d = u / x dot --- when the robot is moving at a constant speed
    # u might need to be the PWM motor value scaled down to [0..1]

# m = (-d * t_90%) / ln(1 - 0.9)

# A = np.array([ [0, 1], [0, -d/m] ])
# B = np.array([ 0, 1/m ])

# sigma values
# sigma_1 and sigma_2: process noise - trust in modeled position (sigma 1) and speed (sigma 2)
    # start with sigma1 = 35 mm and sigma2 = 71 mm/s (or 30 mm/s)
        # Every time the robot travels for 1 second, the ToF measurement is 35 mm off
            # Correlated with how fast the robot runs - use smaller values are larger speeds
# sigma_4: measurement noise --- LARGER VALUE MEANS THERE IS MORE SENSOR ERROR (trusts the sensor data less)
    # error of the sensor: 10-30 mm
    # sigma4 = square root of value in 10-30

In [ ]:
sig_u=np.array([[sigma_1**2,0],[0,sigma_2**2]]) # We assume uncorrelated noise, therefore a diagonal matrix works.
sig_z=np.array([[sigma_4**2]])

In [ ]:
def performKF(data, A, B, n = 2):
    x = np.array([[sensorVal],[0]])
    
    Ad = np.eye(n) + Delta_T * A  # n is the dimension of your state space 
    Bd = Delta_t * B
    
    # Use numpy linspace and interp to make data the same size (if needed)
    
    # loop over data and perform KF
    for val in data:
        sensorVal = val[0]
        timeVal = val[1]
        
        # STILL NEED TO ADD: sigma, y
        kf(timeVal, sigma, sensorVal, y, Ad, Bd)


# C = 1 -> measure positive distance from the wall (initial state)
def kf(mu, sigma, u, y, A, B, C = np.array([[1, 0]])):

    mu_p = A.dot(mu) + B.dot(u) 
    sigma_p = A.dot(sigma.dot(A.transpose())) + Sigma_u
    
    y_m = y-C.dot(mu_p)
    sigma_m = C.dot(sigma_p.dot(C.transpose())) + Sigma_n
    kkf_gain = sigma_p.dot(C.transpose().dot(np.linalg.inv(sigma_m)))

    mu = mu_p + kkf_gain.dot(y_m)    
    sigma=(np.eye(3)-kkf_gain.dot(C)).dot(sigma_p)

    return mu,sigma

In [ ]:


### Move robot forward for 3 secs and get sensor readings ###
rc.get_imu()

rc.move_forward(50)
log.info("IMU Reading: " + str(rc.latest_imu_reading))
await time.sleep(3)
rc.stop()

rc.get_imu()
log.info("IMU Reading: " + str(rc.latest_imu_reading))

<hr>

### There are two possible approaches to reading values from the Artemis board
**Approach 1:** Read values explicitly <br>
**Approach 2:** Notifications

#### You can use a combination of both i.e read some values explicitly and activate notifications for others.

## Approach 1: Read values explicitly
Below is a possible structure to run your robot commands in a loop and explicitly read the sensor values as required. 

You have more control of your code in this approach. However, the read functions could perform slower in comparison to notify events.

In [ ]:
# Add this to the top most cell containing the imports
import asyncio


while True:
    ###### Your code ######
    # Ex: Move the robot for 1 sec
    # rc.move_forward(50)
    # await asyncio.sleep(1)
    # rc.stop()
    
    ###### Read values ######
    # rc.get_imu()
    # rc.get_front_tof()

## Approach 2: Notifications
Below is a possible structure to run your robot command in a loop and utilize the notify events. 

You may have to tweak the sleep time based on how you program your arduino, system OS, system specs, system load and, the number and frequency of notify events. 

However, you **don't have to worry too much** about this if you keep your notify callback functions light. It becomes an issue only when you have a large number of notify events within a (very) short duration of time.

In [ ]:
while True:
    ###### Your code ######
    # Ex: Move the robot for 1 sec
    # rc.move_forward(50)
    # await asyncio.sleep(1)
    # rc.stop()
    
    # Process notify events
    # Sleep the current execution so that the notify callback functions can run
    # The sleep time can be much smaller, but it depends on the OS, hardware specs, number of notify events and the system load
    await asyncio.sleep(0.1)